In [1]:
# Import libraries
import pandas as pd 
import time 
from selenium import webdriver
from selenium.webdriver.support.select import Select 
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException 

### Crawl Data

In [2]:
# Declare browser 
driver = webdriver.Chrome('chromedriver.exe')

# Open url 
driver.get('https://finance.yahoo.com/')
driver.maximize_window()
time.sleep(1)
#sleep(random.randint(1,5))

stock_name = driver.find_element(By.XPATH,'//form[@class="Pos(r)"]/input')
stock_name.click()
stock_name.send_keys('PG')
time.sleep(1)

search = driver.find_element(By.XPATH, '//form[@class="Pos(r)"]/button[2]')
search.click()
time.sleep(1)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13272\989879175.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [3]:
historical_data = driver.find_element(By.XPATH,'//div[@id="quote-nav"]/ul/li[5]/a/span')
historical_data.click()
time.sleep(1)

time_period = driver.find_element(By.XPATH,'//div[@class="Pos(r) D(ib) C($linkColor) Cur(p)"]/span')
time_period.click()
time.sleep(1)

StartDate = driver.find_element(By.XPATH,'//input[@name="startDate"]')
StartDate.click()
time.sleep(1)

t = driver.find_element(By.XPATH,'//div[@class="W(195px)"]/ul[2]/li[3]/button/span')
t.click()

apply =driver.find_element(By.XPATH,'//div[@class="Bgc($lv1BgColor) Bdrs(3px) P(10px)"]/button/span')
apply.click()
time.sleep(1)

In [4]:
# Scroll down to the bottom of the page 
element = driver.find_element(By.TAG_NAME,'body')
i = 1 
while i<65:
    element.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    i+=1 

In [5]:
# Crawl date
date = []
date_list = driver.find_elements(By.XPATH,'//tbody/tr[@class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"]/td[1]')

for i in range(len(date_list)):
    date.append(date_list[i].text)

In [6]:
# Crawl price 
name, price, volume = [], [], []
names = 'PG'

prices = driver.find_elements(By.XPATH,'//tbody/tr[@class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"]/td[5]')
volumes = driver.find_elements(By.XPATH,'//tbody/tr[@class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"]/td[7]')

for i in range(len(prices)):
    name.append(names)
    price.append(prices[i].text)
    volume.append(volumes[i].text)

In [7]:
# Remove duplicate values in date list 
date = pd.unique(date)

In [8]:
# Create dataframe 
df1 = pd.DataFrame(list(zip(name, date, price, volume)), 
                 columns = ['Name', 'Date', 'Price', 'Volume'])
df1.head()

,Name,Date,Price,Volume
0,PG,"Apr 21, 2023",156.07,"14,875,800"
1,PG,"Apr 20, 2023",150.85,"5,370,900"
2,PG,"Apr 19, 2023",151.24,"3,837,600"
3,PG,"Apr 18, 2023",151.21,"4,158,400"
4,PG,"Apr 17, 2023",151.05,"4,296,400"


In [9]:
# Checking data types 
df1.dtypes 

Name      object
Date      object
Price     object
Volume    object
dtype: object

In [10]:
# Replace values 
df1['Volume'] = df1['Volume'].str.replace(',', '')

# Change data type 
df1['Volume'] = df1['Volume'].astype('int64')

In [11]:
# Change data type 
df1['Date'] = df1['Date'].astype('datetime64[ns]')
df1['Price'] = df1['Price'].astype('float64')

# Checking data type 
df1.dtypes

Name              object
Date      datetime64[ns]
Price            float64
Volume             int64
dtype: object

In [12]:
# Export excel data 
df1.to_excel('PgStockData.xlsx',index=False)

I also crawl stock data of Unilever using this method  

In [13]:
# Load dataframe
df2 = pd.read_excel('UlStockData.xlsx')
df2.head()

,Name,Date,Price,Volume
0,UL,2023-04-21,54.51,2535000
1,UL,2023-04-20,54.28,1505600
2,UL,2023-04-19,53.86,1719600
3,UL,2023-04-18,53.78,2735400
4,UL,2023-04-17,53.78,1475300


In [14]:
# Union 2 dataframes
df = pd.concat([df1, df2], ignore_index=True)
df.head()

,Name,Date,Price,Volume
0,PG,2023-04-21,156.07,14875800
1,PG,2023-04-20,150.85,5370900
2,PG,2023-04-19,151.24,3837600
3,PG,2023-04-18,151.21,4158400
4,PG,2023-04-17,151.05,4296400


In [15]:
# Export data 
df.to_excel('StockData.xlsx')